In [ ]:
import pandas as pd
from pathlib import Path
import pymc as pm
import plotly.express as px
import numpy as np
import pandas as pd

from estival.model import BayesianCompartmentalModel
import estival.priors as esp
import estival.targets as est
from estival.wrappers import pymc as epm
from tbdynamics import model
from tbdynamics.utils import round_sigfig
from tbdynamics.inputs import fixed_parameters
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Define variables

In [ ]:
PROJECT_PATH = Path().resolve()
DATA_PATH = PROJECT_PATH / 'data'
pd.options.plotting.backend = "plotly"

time_start = 1700.0
time_end = 2023.0
time_step = 0.1

compartments = [
    "susceptible",
    "early_latent",
    "late_latent",
    "infectious",
    "on_treatment",
    "recovered",
]

latent_compartments = [
    "early_latent",
    "late_latent",
]
infectious_compartments = [
    "infectious",
    "on_treatment",
]


age_strata = [0,5,15,35,50,70] 
matrix = np.ones((6, 6)) # Homo mixing

## Define Model

### Base model

In [ ]:
tb_model = model.build_model(
    compartments,
    latent_compartments,
    infectious_compartments,
    age_strata,
    time_start,
    time_end,
    time_step,
    fixed_parameters,
    matrix
)

### Params and calibration targets

In [ ]:
params = {
    "smear_positive_death_rate":0.364337776897486,
    "smear_negative_death_rate": 0.027588310343242016, 
    "smear_positive_self_recovery": 0.20344728302826143,
    "smear_negative_self_recovery": 0.22723824998716693,
}


priors = [
    esp.UniformPrior("start_population_size", (100000, 30000000)),
    esp.UniformPrior("contact_rate", (0, 10)),  # multiplied with eigenvalue
    esp.UniformPrior("rr_infection_latent", (0.2, 0.5)),
    esp.UniformPrior("rr_infection_recovered", (0.1, 0.5)),
    esp.UniformPrior("progression_multiplier", (1.0, 5.0)),
    esp.UniformPrior("seed_time", [1700.0, 1900.0]),
    esp.UniformPrior("seed_duration", [1.0, 5.0]),
    esp.UniformPrior("seed_num", [1.0, 1000.0]),
    esp.UniformPrior("smear_positive_death_rate", (0.335, 0.449)),
    esp.UniformPrior("smear_negative_death_rate", (0.017, 0.035)),
    esp.UniformPrior("smear_positive_self_recovery", (0.177, 0.288)),
    esp.UniformPrior("smear_negative_self_recovery", (0.073, 0.209)),

]

pop = pd.Series(
    {
        2009: 86025000,
        2019: 96484000,

    }
)

targets = [
    est.NormalTarget("total_population", pop, stdev=100000),
]

calibration_model = BayesianCompartmentalModel(tb_model, params, priors, targets)

### Running Calibration

In [ ]:
with pm.Model() as pmc_model:
    start_params = {k: np.clip(v, *calibration_model.priors[k].bounds(0.99)) for k, v in params.items() if k in calibration_model.priors}
    variables = epm.use_model(calibration_model)
    map_params = pm.find_MAP(start=start_params, vars=variables, maxeval= 1000,include_transformed=False)
    map_params = {k: float(v) for k, v in map_params.items()}
    print('Best calibration parameters found:')
for i_param, param in enumerate(map_params):
    print(f'   {param}: {round_sigfig(map_params[param], 4)} (within bound {priors[i_param].bounds()}')

In [ ]:
params.update(map_params)
tb_model.run(params)
derived_df_0 = tb_model.get_derived_outputs_df()

### Population output

In [ ]:
fig2_1 = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="total_population",
)
fig2_2 = px.scatter(x= pop.index, y = pop.values)
fig2_2.update_traces(marker=dict(color="red"))
fig2_3 = go.Figure(
    data=fig2_1.data + fig2_2.data,
)
fig2_3.update_layout(
    title="Modelled vs Data", title_x=0.5, xaxis_title="Year", yaxis_title="Population"
)
fig2_3.show()

In [ ]:
derived_df_0[[f'total_populationXage_{i}' for i in age_strata]].plot(title='Modelled populatation by age group', kind ='area')

In [ ]:
derived_df_0['percentage_latent'].plot()

In [ ]:
derived_df_0['prevalence_infectious'].plot()

In [ ]:
def plot_faceted_population_chart(df, selected_years, age_groups):
    """
    Plot a faceted chart with each facet for one selected year, arranged in 2 columns.
    Each chart shows the total population for each age group beside the infectious population.
    
    Parameters:
    - df: DataFrame with population data.
    - selected_years: List of years to plot.
    - age_groups: List of age groups.
    """
    # Determine the number of rows needed for the subplots, given 2 columns
    rows = int(np.ceil(len(selected_years) / 2))
    
    # Create subplots with 2 columns
    fig = make_subplots(rows=rows, cols=2,
                        subplot_titles=[f'Year {year}' for year in selected_years],
                        shared_xaxes=True)
    
    total_legend_shown = False
    latent_legend_shown = False
    
    # Adjust subplot indexing for 2-column layout
    subplot_index = [(i, j) for i in range(1, rows + 1) for j in range(1, 3)]

    for count, year in enumerate(selected_years):
        # Calculate row and col based on count
        row, col = subplot_index[count]
        year_df = df.loc[round(year,1)]  # Adjust this line as necessary for your DataFrame's index

        for age in age_groups:
            total_pop = year_df[f'total_populationXage_{age}']
            latent_pop = year_df[f'latent_population_sizeXage_{age}']
            
            # Total population bar
            fig.add_trace(go.Bar(name='Total Population', x=[f'Age {age}'], y=[total_pop],
                                 marker=dict(color='rgba(100, 150, 240, 0.6)'),
                                 legendgroup='Total', showlegend=not total_legend_shown), row=row, col=col)
            
            # Infectious population bar, plotted beside the total population
            fig.add_trace(go.Bar(name='Infectious Population', x=[f'Age {age}'], y=[latent_pop],
                                 marker=dict(color='rgba(255, 100, 100, 0.6)'),
                                 legendgroup='Latent', showlegend=not latent_legend_shown), row=row, col=col)
            
            if not total_legend_shown:
                total_legend_shown = True
            if not latent_legend_shown:
                latent_legend_shown = True

    # Update layout to make bars thicker and set the ticks for each age group
    fig.update_layout(height=300*rows, width=700,
                      title_text="Total vs Latent Population by Age Group and Year",
                      barmode='group',
                      xaxis_tickangle=-45,
                      legend=dict(
                          orientation="h",
                          yanchor="bottom",
                          y=1.02,
                          xanchor="right",
                          x=1
                      ),
                      bargap=0.005, # Reduce gap between bars to make them thicker
                      )
    
    fig.show()

In [ ]:
plot_faceted_population_chart(derived_df_0, np.arange(1990,2020,5), age_strata)